In [1]:
from sentence_transformers import SentenceTransformer, util
# import mammoth # THIS IS FOR CONVERTING THE DOCX TO MD
from os import path, listdir
import re
import itertools
from urllib.request import urlopen

In [2]:
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

In [3]:
%run load_docx.ipynb

In [4]:
source = 'Solution_mappers'

In [5]:
directory = f'./data/local/{source}'

In [6]:
# file = '../Explorer_BGD.docx'

In [27]:
files = [f for f in listdir(directory) if f != '.DS_Store']

In [8]:
def embed (string):
    return model.encode(string)

In [9]:
def embedDocs (file):
    file_name, file_extension = path.splitext(file)
    if file_extension == '.docx':
        md = processMD(path.join(directory, file))
    doc = cleanMD(md.value).split('\n\n')
    doc_embeddings = [(s,embed(s)) for s in doc]
    return (file_name, doc_embeddings)

In [10]:
for f in files[:1]:
    file_name, embeddings = embedDocs(f)

In [11]:
# doc = md.value.split('\n\n')

In [12]:
# doc_embeddings = [(s,embed(s)) for s in doc]

In [13]:
def getType (md, w):
    principle = 'Purpose/ reason: Why is this important?'
    tactic = r'Tactic description: When do you apply this tactic\? How does it work\?'
    skill = r'Skills description: What does the ability help you do, achieve, or produce\?'
    tool = r'(Type of tool: If it is a tool, what type of tool is it\?|Short description: Describe in one sentence what the tool\/ method is, and what it does\.|Description of the tool: what is it\?)'

    res_principles = len(re.findall(principle, md))
    res_tactics = len(re.findall(tactic, md))
    res_skills = len(re.findall(skill, md))
    res_tools = len(re.findall(tool, md))

    # if res_principles == 0 and res_tactics == 0 and res_skills == 0 and res_tools == 0:
    print(w)
    print('pinciples', res_principles)
    print('tactics', res_tactics)
    print('skills', res_skills)
    print('tools', res_tools)
    print('\n')

    l = [('principles', res_principles), ('tactics', res_tactics), ('skills', res_skills), ('tools', res_tools)]
    return [t for (t,c) in l if c == max(res_principles, res_tactics, res_skills, res_tools)]

In [14]:
def filterMD (md):
    # REMOVE ALL THE CHECKLISTS
    md = re.sub(r'\#\#\#\# [\w\s\:\?\/\(\)\\;&]+\n', '', md)
    md = re.sub(r'\#+', '', md)
    md = re.sub(r'\n-[\s_]+(Sense|Explore|Develop|Test|Diffuse|Catalyze)', '', md)
    md = re.sub(r'\n-[\s_]+(Exploration|Solutions mapping|Experimentation)', '', md)
    md = re.sub(r'\n-[\s_]+(The advocate|The analyst|The communicator|The convener|The hustler|The project manager|The qualitative researcher|The visionary)', '', md)
    md = re.sub(r'\n-[\s_]+(Framework, Model|Guide, Manual|Platform, Portal|Playbook, Toolkit|Worksheet, Canvas|Process|Pilots|Proof of concept|Prototyping)', '', md)
    md = re.sub(r'\n-[\s_]+(Yes|No)', '', md)
    md = re.sub(r'[_\\]+', ' ', md)
    md = re.sub(r'\s+', ' ', md)
    return md.strip()

In [15]:
fromDB = False

In [16]:
if fromDB == True:
    %run get_keywords.ipynb
else:
    source = '../elements/__pages__/'
    seed_keywords = [(path.splitext(f)[0], readFile(path.join(source, f))) for f in listdir(source) if f != '.DS_Store']
    seed_keywords = [(w, filterMD(str(b)), getType(str(b), w)[0]) for (w, b) in seed_keywords]

NESTA DIY Toolkit
pinciples 0
tactics 0
skills 0
tools 1


Transformative leadership skills and decision making
pinciples 0
tactics 0
skills 5
tools 0


EGAP Pre-analysis plans
pinciples 0
tactics 0
skills 0
tools 2


ArcGIS
pinciples 0
tactics 0
skills 0
tools 4


Collaboration
pinciples 2
tactics 0
skills 0
tools 0


Self-awareness
pinciples 1
tactics 0
skills 0
tools 0


Urbanist AI
pinciples 0
tactics 0
skills 0
tools 1


Setting up the first block of the pyramid
pinciples 0
tactics 1
skills 0
tools 0


Transparency and Honesty
pinciples 2
tactics 0
skills 0
tools 0


Narrative analysis
pinciples 0
tactics 0
skills 0
tools 1


Trust
pinciples 3
tactics 0
skills 0
tools 0


Open Data Kit
pinciples 0
tactics 0
skills 0
tools 1


Information management
pinciples 0
tactics 0
skills 1
tools 0


Co-design
pinciples 0
tactics 1
skills 0
tools 0


Design lens
pinciples 0
tactics 0
skills 0
tools 1


Flexibility and adaptability
pinciples 2
tactics 0
skills 0
tools 0


Design thinking
pinci

In [17]:
# print([(w,t) for (w,b,t) in seed_keywords])

In [18]:
for w,b,t in seed_keywords:
    if len(b) < 50:
        print(w, len(b))

seed_keywords = [(w,b,t) for (w,b,t) in seed_keywords if len(b) > 50]

In [19]:
seed_keywords[11]

('Open Data Kit',
 'b"[[type:tools]] n n ODK (Open Data Kit) is a powerful open-source software platform that enable xe2 x80 xa6 n n Image: What does it look like? n n n Reference: Where can we find more information or download materials? n n n<a>https://getodk.org/</a> n n Description: What is it? (max 1,500 characters) n n n nODK (Open Data Kit) is a powerful open-source software platform that enables researchers, field teams, and other professionals to collect data efficiently, both online and offline. Here are the key features and benefits of ODK: n nForm Building and Customization: n nODK allows you to build powerful forms using the XLSForm standard in tools like Excel or Google Sheets. n nYou can include various form elements such as photos, GPS locations, skip logic, calculations, external datasets, and support for multiple languages. n nOnline and Offline Data Collection: n nWhether you xe2 x80 x99re in the field or the office, ODK works seamlessly. You can use either the mobil

In [20]:
embedTitle = False

if embedTitle:
    keyword_embeddings = [(w,t,embed(w)) for (w,b,t) in seed_keywords]
else:
    keyword_embeddings = [(w,t,embed(b)) for (w,b,t) in seed_keywords]

In [21]:
def compare (e1, e2):
    return util.pytorch_cos_sim(e1, e2).tolist()[0][0]

In [22]:
# Get the list of countries and iso3 codes ("countries" dataframe)
%run get_countries.ipynb

def getAudioTag (snippet):
    if snippet.startswith('# '):
        snippet = re.sub(r'^#\s+', '', snippet, flags=re.MULTILINE)
        role, country, actor = [re.sub(r'\\', '', s.strip(), flags=re.MULTILINE) for s in snippet.split('-', 2)]
        print(role, country, actor)
        iso3 = pd.unique(countries.loc[countries['name'] == country.strip()]['iso3'])[0]
        continent = pd.unique(countries.loc[countries['name'] == country.strip()]['continent'])[0]
        # [[audio:https://acclabplatforms.blob.core.windows.net/rndarchive/SolutionMappers/SolutionMappers_BIH.m4a]]
        baseurl = 'https://acclabplatforms.blob.core.windows.net/rndarchive/'
        route_options = []
        for s in role.split(' '):
            sub_options = []
            s = s.strip().capitalize()
            sub_options.append(s)
            if s.endswith('s'):
                sub_options.append(s[:-1])
            else:
                sub_options.append(f'{s}s')
            route_options.append(sub_options)
        routes = [''.join(r) for r in list(itertools.product(*route_options))]
        audio_tag = None
        valid_role = routes[0]
        for r in routes:
            try:
                url = f'{baseurl}{r}/{r}_{iso3}.m4a' 
                code = urlopen(url).getcode()
                if code == 200:
                    audio_tag = f'[[audio:{url}]]'
                    valid_role = r
                else:
                    pass
            except:
                pass
        return (audio_tag, iso3, country, continent, valid_role)
        

/var/folders/tg/c_k1syfs2b9gvpkyjkcmdqx00000gn/T/ipykernel_14668/1832143831.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  countries = pd.read_sql_query(query,con=conn)


In [23]:
def generateOutput (doc_embeddings):
    topTags = []
    useTypes = False
    
    output_string = ''
    output_name = None
    
    for i, snippet in enumerate(doc_embeddings):
        if len(snippet[0]) > 500 and not snippet[0].startswith('#'):
            distances = [(w,t,compare(e, snippet[1])) for (w,t,e) in keyword_embeddings]
            distances.sort(key = lambda tup: tup[2], reverse = True)
            types = list(set([t for (w,t,e) in keyword_embeddings]))
            
            if useTypes:
                for t in types:
                    top_keyword = [w for (w, dt, e) in distances if dt == t][0]
                    keyword, extension = path.splitext(top_keyword)
                    output_string = output_string + "[[{}:{}]]".format(t.replace('capture card', '').strip().lower(), keyword)
                    output_string = output_string + '\n'
            else:
                # print([(w, e) for (w, dt, e) in distances][0:3])
                top_keyword = [(w, dt) for (w, dt, e) in distances][:4]
                # output_string = output_string + "[[{}]]".format(top_keyword)
                for w, dt in top_keyword:
                    keyword, extension = path.splitext(w)
                    output_string = output_string + "[[{}:{}]]".format(dt, keyword)
                    output_string = output_string + '\n'
                # output_string = output_string + '\n\n'
    
            output_string = output_string + snippet[0]
            output_string = output_string + '\n\n'
            
            topTags += [w for (w,t,e) in distances[:5]]
            output_string = output_string + '\n'
            # print(snippet[0], distances[:5])
        else:
            if i == 0: 
                print(snippet[0])
                audio_tag, iso3, country, continent, valid_role = getAudioTag(snippet[0])
                if audio_tag is not None:
                    output_string = output_string + audio_tag
                    output_string = output_string + '\n\n'
                if continent is not None:
                    output_string = output_string + f'[[continent:{continent}]]'
                    output_string = output_string + '\n'
                if country is not None:
                    output_string = output_string + f'[[country:{country}]]'
                    output_string = output_string + '\n\n'
                if iso3 is not None and valid_role is not None:
                    output_name = f'{valid_role}_{iso3}.md'
            output_string = output_string + snippet[0]
            output_string = output_string + '\n\n'
    return (output_string, topTags, output_name)

In [24]:
def findDuplicates (l):
    seen = set()
    duplicates = []

    for x in l:
        if x in seen and x not in duplicates:
            duplicates.append(x)
        else:
            seen.add(x)
    return duplicates

In [29]:
for f in files:
    print(f)
    file_name, embeddings = embedDocs(f)
    output_string, topTags, output_name = generateOutput(embeddings)

    print (findDuplicates(topTags))
    if output_name is None:
        output_name = "{}.md".format('_'.join([t for t in file_name.split('/') if t != '..' and t != '.']))
    print(output_name)
    output_file = open("../stories/__pages__/{}".format(output_name), 'w')
    output_file.write(output_string)
    output_file.close()
    print('file written')
    print('\n')
    
# outputs = [generateOutput(embedDocs(f)) for f in files]

Paulina_edited-en-US.docx
# Solutions Mapper\-Ecuador\-Paulina Jimenez
Solutions Mapper Ecuador Paulina Jimenez
['Design', 'Curiosity', 'Creating space for emerging themes', 'Human centering and adaptability', 'Recognize local expertise', 'Systems thinking', 'Inclusivity and diversity', 'Trust', 'Self-awareness', 'Recognize and promote agency', 'Human centered design']
SolutionMappers_ECU.md
file written


Amina_edited-en-US.docx
# Solutions Mapper\-Bosnia and Herzegovina\-Amina Omicevic
Solutions Mapper Bosnia and Herzegovina Amina Omicevic
['Recognize local expertise', 'Systems thinking', 'Getting in there', 'Design', 'Recognize and promote agency', 'Always increase social capital', 'Human centering and adaptability', 'GIS systems']
SolutionMappers_BIH.md
file written


Gabriel_edited-en-US.docx
# Solutions Mapper\-Peru\-Gabriel Lama
Solutions Mapper Peru Gabriel Lama
['Adaptability', 'Cultural adaptability', 'Creativity', 'Systems thinking', 'Recognize local expertise', 'Desiging th